In [ ]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing  import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR, SVC

In [ ]:
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform
import matplotlib.pyplot as plt

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")


In [ ]:
df_merge= pd.read_csv('../../Data/날씨+일별주차이용.csv')

In [ ]:
pd.read_csv('../../Data/한강2403-2409.csv')
df = pd.read_csv('../../Data/서울시 한강공원 월별 이용객 현황.csv',encoding='euc-kr')


In [ ]:

print(df.columns)
df = df.loc[:,['공원 코드','공원명','일반이용자(아침)','일반이용자(낮)', '일반이용자(저녁)','현황 일시']]

In [ ]:
df['현황 일시']= pd.to_datetime(df['현황 일시'])

In [ ]:
df_merge

In [ ]:
# PLT-001: 강서   
# PLT-002: 난지   
# PLT-003: 망원   
# PLT-004: 이촌   
# PLT-005: 뚝섬       
# PLT-006: 광나루     
# PLT-007: 양화   
# PLT-008: 여의도     
# PLT-009: 반포   
# PLT-0010: 잠원  
# PLT-0011: 잠실
                             
# Hzone011: 강서  
# Hzone009: 난지  
# Hzone010: 망원  
# Hzone006: 이촌  
# Hzone003: 뚝섬  
# Hzone001 광나루     
# Hzone008: 양화  
# Hzone007: 여의도    
# Hzone005: 반포  
# Hzone004: 잠원  
# Hzone002: 잠실  

In [ ]:
# 주차장일별['code'] 
codelist = []
for code in df['공원 코드']:
    codelist.append(code.replace('Hzone',""))

In [ ]:
df['code'] = codelist

In [ ]:
df['code']= pd.to_numeric(df['code'])

In [ ]:
# df.iloc[:,]
df_merge['날짜']= pd.to_datetime(df_merge['날짜'])

In [ ]:
df

In [ ]:
df['일반이용자(아침)'].replace(0,np.nan, inplace=True)
df['일반이용자(저녁)'].replace(0,np.nan,inplace=True)
df['일반이용자(낮)'].replace(0,np.nan,inplace=True)


In [ ]:
df.info()

In [ ]:
df.rename(columns={'현황 일시' : '날짜'},inplace=True)

In [ ]:
df['년도'] = df['날짜'].dt.year
df['월'] = df['날짜'].dt.month

In [ ]:
df

In [ ]:
nanmean= pd.DataFrame({'아침 평균':df.groupby(['code','월'])['일반이용자(아침)'].mean(),
'낮 평균':df.groupby(['code','월'])['일반이용자(낮)'].mean(),
'저녁 평균':df.groupby(['code','월'])['일반이용자(저녁)'].mean()},
)

In [ ]:
nanmean.reset_index(inplace=True)

In [ ]:
nanmean['합계'] = nanmean['아침 평균'] + nanmean['낮 평균'] + nanmean['저녁 평균']

In [ ]:
nanmean['아침 비율'] = nanmean['아침 평균']/nanmean['합계']
nanmean['저녁 비율'] = nanmean['저녁 평균']/nanmean['합계']
nanmean['낮 비율'] = nanmean['낮 평균']/nanmean['합계']

In [ ]:
nanmean.drop('합계', axis=1, inplace= True)

In [ ]:
nanmean[nanmean['code'] ==5]

In [ ]:
df = pd.merge(df,nanmean, on=['code', '월'])


In [ ]:
df['월'].unique()

In [ ]:
df['일반이용자(아침)'].fillna(df['아침 평균'], inplace=True)
df['일반이용자(낮)'].fillna(df['낮 평균'], inplace=True)
df['일반이용자(저녁)'].fillna(df['저녁 평균'], inplace=True)


In [ ]:
df

In [ ]:
df

In [ ]:
df.drop(columns=['날짜','공원명','공원 코드','아침 평균','낮 평균','저녁 평균'],axis=1,inplace=True)

--------

In [ ]:
df_merge3= pd.merge(df_merge, df, how='left', left_on=['code','월','년도'], right_on=['code','월','년도'])

In [ ]:
df_merge3['휴일여부']= df_merge3['휴일여부'].map({'주말' : 1, '평일' : 0, '휴일' : 1})

In [ ]:
df_merge3.drop(['최저기온 시각(hhmi)','최고기온 시각(hhmi)','평균 이슬점온도(°C)','평균 상대습도(%)'], axis=1, inplace= True)

In [ ]:
result  = []
for i in df_merge3['일강수량(mm)']:
    if i == 0 :
        result.append(0)
    else:
        result.append(1)

In [ ]:
df_merge3['비'] = result

In [ ]:
df_merge3

In [ ]:
code_month_mean=pd.DataFrame({'지점월별아침평균':df_merge3.groupby(['code','월','휴일여부'])['일반이용자(아침)'].mean(),'지점월별낮평균':df_merge3.groupby(['code','월','휴일여부'])['일반이용자(낮)'].mean(),'지점월별저녁평균':df_merge3.groupby(['code','월','휴일여부'])['일반이용자(저녁)'].mean()})
code_month_mean.reset_index(inplace=True)

In [ ]:
code_month_mean

In [ ]:
df_merge3=pd.merge(df_merge3,code_month_mean, on=['code','월', '휴일여부'])

In [ ]:
df_merge3['아침 비율'].fillna(df_merge3['지점월별아침평균']/(df_merge3.iloc[:,-3:].sum(axis=1)), inplace=True)
df_merge3['낮 비율'].fillna(df_merge3['지점월별낮평균']/(df_merge3.iloc[:,-3:].sum(axis=1)), inplace=True)
df_merge3['저녁 비율'].fillna(df_merge3['지점월별저녁평균']/(df_merge3.iloc[:,-3:].sum(axis=1)), inplace=True)

In [ ]:
df_merge3['일반이용자(아침)'].fillna(df_merge3['지점월별아침평균'],inplace=True)
df_merge3['일반이용자(낮)'].fillna(df_merge3['지점월별낮평균'],inplace=True)
df_merge3['일반이용자(저녁)'].fillna(df_merge3['지점월별저녁평균'],inplace=True)

------------
-------------

In [ ]:
# 파일 저장
# df_merge3.to_csv('../../Data/version1.csv',index=False)

code = 지구별에서 숫자값만 추출
지구별
PLT-001: 강서   
PLT-002: 난지   
PLT-003: 망원   
PLT-004: 이촌   
PLT-005: 뚝섬       
PLT-006: 광나루     
PLT-007: 양화   
PLT-008: 여의도     
PLT-009: 반포   
PLT-0010: 잠원  
PLT-0011: 잠실  

In [ ]:
df = pd.read_csv('../../Data/version1.csv')
df.head()

In [ ]:
df

In [ ]:
# df.drop('일',axis=1, inplace= True)

In [ ]:
df.head()

In [ ]:

# 상관계수 계산
correlation_matrix = df.iloc[:,1:].corr()

# 그림 크기 설정
plt.figure(figsize=(10, 8))

# heatmap 생성
sns.heatmap(correlation_matrix, 
            annot=True,  # 각 셀에 숫자 표시
            cmap='coolwarm',  # 색상 맵 설정
            vmin=-1, vmax=1,  # 색상 범위 설정
            center=0,  # 0을 중심으로 색상 대칭
            square=True,  # 정사각형 셀
            fmt='.2f')  # 소수점 둘째 자리까지 표시

# 제목 설정
plt.title('Correlation Heatmap', fontsize=16)

# 그래프 표시
plt.show()


In [ ]:
# df['아침이용비율'] =
df['아침이용비율']= df['일반이용자(아침)'] / (df['일반이용자(아침)']+df['일반이용자(낮)']+df['일반이용자(저녁)'])
df['낮이용비율']= df['일반이용자(낮)'] / (df['일반이용자(아침)']+df['일반이용자(낮)']+df['일반이용자(저녁)'])
df['저녁이용비율']= df['일반이용자(저녁)'] / (df['일반이용자(아침)']+df['일반이용자(낮)']+df['일반이용자(저녁)'])
# df

In [ ]:
# df['아침주차']= df['주차대수'] * df['아침이용비율']
# df['낮주차']= df['주차대수'] * df['낮이용비율']
# df['저녁주차']= df['주차대수'] * df['저녁이용비율']
# df.head()

In [11]:
park_info=pd.read_csv('../../Data/한강공원 주차장 정보.csv',encoding= 'euc-kr')

In [12]:
park_info.지구별.unique()

array(['강서', '광나루', '난지', '뚝섬', '망원', '반포', '양화', '여의도', '이촌', '잠실', '잠원'],
      dtype=object)

In [14]:
park_info.head()

,지구별,주차장별,위치정보(위도),위치정보(경도),주소,면수,장애면수,정기면수,주중 이용시간 시작,주중 이용시간 종료,...,간격시간,간격요금,전일요금,주간요금,야간요금,정기금금액,주차장 운영업체,업체 전화번호,운영 시작기간,운영 종료기간
0,강서,강서1주차장,37.585709,126.818279,서울강서구방화동61(방화동),53,0,0,06:00,21:00,...,10,200,10000,0,0,50000,공우이엔씨(주)_양화_강서,02-2068-9617,2023-12-01 00:00:00.0,2025-11-30 00:00:00.0
1,광나루,"광나루1,2주차장",37.553885,127.123160,서울강동구암사동613-11,168,0,0,06:00,21:00,...,10,200,10000,0,0,50000,공우이엔씨(주)_잠실_광나루,02-3431-3774,2023-12-01 00:00:00.0,2025-11-30 00:00:00.0
2,광나루,광나루3주차장,37.543078,127.117775,서울강동구풍납동49-2,191,0,0,06:00,21:00,...,10,200,10000,0,0,50000,공우이엔씨(주)_잠실_광나루,02-3431-3774,2023-12-01 00:00:00.0,2025-11-30 00:00:00.0
3,난지,"난지1,2,3주차장",37.569822,126.873640,서울마포구상암동487-370,544,0,0,06:00,21:00,...,10,200,10000,0,0,50000,(주)씨앤에스자산관리_망원_난지,070-7778-6678,2023-02-01 00:00:00.0,2025-01-31 00:00:00.0
4,뚝섬,뚝섬1주차장,37.527691,127.078163,서울광진구자양동409(자양동),67,0,0,06:00,21:00,...,10,200,10000,0,0,50000,공우이엔씨(주)_뚝섬,02-455-7276,2023-12-01 00:00:00.0,2024-11-30 00:00:00.0


In [17]:
park_info.head()

,지구별,주차장별,위치정보(위도),위치정보(경도),주소,면수,장애면수,정기면수,주중 이용시간 시작,주중 이용시간 종료,...,간격시간,간격요금,전일요금,주간요금,야간요금,정기금금액,주차장 운영업체,업체 전화번호,운영 시작기간,운영 종료기간
0,강서,강서1주차장,37.585709,126.818279,서울강서구방화동61(방화동),53,0,0,06:00,21:00,...,10,200,10000,0,0,50000,공우이엔씨(주)_양화_강서,02-2068-9617,2023-12-01 00:00:00.0,2025-11-30 00:00:00.0
1,광나루,"광나루1,2주차장",37.553885,127.123160,서울강동구암사동613-11,168,0,0,06:00,21:00,...,10,200,10000,0,0,50000,공우이엔씨(주)_잠실_광나루,02-3431-3774,2023-12-01 00:00:00.0,2025-11-30 00:00:00.0
2,광나루,광나루3주차장,37.543078,127.117775,서울강동구풍납동49-2,191,0,0,06:00,21:00,...,10,200,10000,0,0,50000,공우이엔씨(주)_잠실_광나루,02-3431-3774,2023-12-01 00:00:00.0,2025-11-30 00:00:00.0
3,난지,"난지1,2,3주차장",37.569822,126.873640,서울마포구상암동487-370,544,0,0,06:00,21:00,...,10,200,10000,0,0,50000,(주)씨앤에스자산관리_망원_난지,070-7778-6678,2023-02-01 00:00:00.0,2025-01-31 00:00:00.0
4,뚝섬,뚝섬1주차장,37.527691,127.078163,서울광진구자양동409(자양동),67,0,0,06:00,21:00,...,10,200,10000,0,0,50000,공우이엔씨(주)_뚝섬,02-455-7276,2023-12-01 00:00:00.0,2024-11-30 00:00:00.0


In [16]:
df.head()

,주차장명,주차대수,이용시간,code,월,일,휴일여부,평균기온(°C),일반이용자(아침),일반이용자(낮),일반이용자(저녁),비,아침이용비율,낮이용비율,저녁이용비율
0,뚝섬1주차장,273,1.397497,5,11,10,1,14.4,5489.393939,39916.454545,56486.666667,0,0.053874,0.391751,0.554375
1,잠실1주차장,1059,1.044397,11,11,10,1,14.4,6577.571429,30023.551020,24601.612245,0,0.107472,0.490559,0.401969
2,강서1주차장,329,1.361449,1,11,10,1,14.4,10556.060606,46010.606061,29326.363636,0,0.122898,0.535673,0.341429
3,여의도5주차장,229,1.638210,8,11,10,1,14.4,4180.727273,58907.515152,57783.333333,0,0.034588,0.487356,0.478056
4,여의도2주차장,1934,1.293821,8,11,10,1,14.4,4180.727273,58907.515152,57783.333333,0,0.034588,0.487356,0.478056


In [18]:
df=pd.merge(df, park_info.loc[:,['주차장별','면수']], how='inner', left_on='주차장명', right_on='주차장별')

In [19]:
df.drop('주차장별',axis=1, inplace= True)

In [ ]:
# df['아침주차비율'] = df['면수']*df['아침이용비율'] 
# df['낮주차비율'] = df['면수']*df['낮이용비율']
# df['저녁주차비율'] = df['면수']*df['저녁이용비율']
# df.head()

In [ ]:

# 그림 크기 설정
plt.figure(figsize=(10, 8))

sns.heatmap(
            df.loc[:,['주차대수','낮이용비율']].corr(),
            annot=True,  # 각 셀에 숫자 표시
            cmap='coolwarm',  # 색상 맵 설정
            vmin=-1, vmax=1,  # 색상 범위 설정
            center=0,  # 0을 중심으로 색상 대칭
            square=True,  # 정사각형 셀
            fmt='.2f')  # 소수점 둘째 자리까지 표시

# 제목 설정
plt.title('Correlation Heatmap', fontsize=16)

# 그래프 표시
plt.show()


In [20]:
df.head()

,주차장명,주차대수,이용시간,code,월,일,휴일여부,평균기온(°C),일반이용자(아침),일반이용자(낮),일반이용자(저녁),비,아침이용비율,낮이용비율,저녁이용비율,면수
0,뚝섬1주차장,273,1.397497,5,11,10,1,14.4,5489.393939,39916.454545,56486.666667,0,0.053874,0.391751,0.554375,67
1,잠실1주차장,1059,1.044397,11,11,10,1,14.4,6577.571429,30023.551020,24601.612245,0,0.107472,0.490559,0.401969,471
2,강서1주차장,329,1.361449,1,11,10,1,14.4,10556.060606,46010.606061,29326.363636,0,0.122898,0.535673,0.341429,53
3,여의도5주차장,229,1.638210,8,11,10,1,14.4,4180.727273,58907.515152,57783.333333,0,0.034588,0.487356,0.478056,217
4,여의도2주차장,1934,1.293821,8,11,10,1,14.4,4180.727273,58907.515152,57783.333333,0,0.034588,0.487356,0.478056,171


In [22]:
grouped_df = df.groupby(['code', '월','휴일여부'], as_index=False)['주차대수'].sum()
grouped_df

,code,월,휴일여부,주차대수
0,1,1,0,9256
1,1,1,1,9291
2,1,2,0,10356
3,1,2,1,10663
4,1,3,0,16484
...,...,...,...,...
259,11,10,1,95461
260,11,11,0,200544
261,11,11,1,48306
262,11,12,0,129930


In [23]:
test=pd.merge(grouped_df, df.loc[:,['주차대수','code','월','주차장명','휴일여부']], how='right', left_on=['code','월','휴일여부'], right_on=['code','월','휴일여부'])

In [24]:
test

,code,월,휴일여부,주차대수_x,주차대수_y,주차장명
0,5,11,1,83614,273,뚝섬1주차장
1,11,11,1,48306,1059,잠실1주차장
2,1,11,1,10128,329,강서1주차장
3,8,11,1,181766,229,여의도5주차장
4,8,11,1,181766,1934,여의도2주차장
...,...,...,...,...,...,...
39780,8,1,0,313354,873,여의도1주차장
39781,1,1,0,9256,28,강서1주차장
39782,8,1,0,313354,114,여의도1주차장
39783,1,1,0,9256,12,강서1주차장


In [25]:
test['주차비율'] = (test['주차대수_y']/test['주차대수_x'])

In [26]:
test

,code,월,휴일여부,주차대수_x,주차대수_y,주차장명,주차비율
0,5,11,1,83614,273,뚝섬1주차장,0.003265
1,11,11,1,48306,1059,잠실1주차장,0.021923
2,1,11,1,10128,329,강서1주차장,0.032484
3,8,11,1,181766,229,여의도5주차장,0.001260
4,8,11,1,181766,1934,여의도2주차장,0.010640
...,...,...,...,...,...,...,...
39780,8,1,0,313354,873,여의도1주차장,0.002786
39781,1,1,0,9256,28,강서1주차장,0.003025
39782,8,1,0,313354,114,여의도1주차장,0.000364
39783,1,1,0,9256,12,강서1주차장,0.001296


In [38]:
df['주차비율'] = test['주차비율']

In [43]:
df.rename(columns={'아침이용비율' : '월별아침이용자비율', '낮이용비율' : '월별낮이용자비율', '저녁이용비율' : '월별저녁이용자비율'},inplace= True)

In [56]:
# df['주차대수']*df['월별아침이용자비율']
# df['일반이용자(아침)']*df['주차비율']
df['아침이용자']=df['주차비율'] * df['일반이용자(아침)']
df['낮이용자']=df['주차비율'] * df['일반이용자(낮)']
df['저녁이용자']=df['주차비율'] * df['일반이용자(저녁)']

In [ ]:
df.drop(['일반이용자(아침)','일반이용자(낮)','일반이용자(저녁)'],axis=1,inplace= True)

In [65]:
df.drop(['월별아침이용자비율','월별낮이용자비율','월별저녁이용자비율'],axis=1, inplace= True)

> 주차비율 = 각 주차장 주차대수 / 월별code 주차대수 합

In [66]:
df

,주차장명,주차대수,이용시간,code,월,일,휴일여부,평균기온(°C),비,면수,주차비율,아침이용자,낮이용자,저녁이용자
0,뚝섬1주차장,273,1.397497,5,11,10,1,14.4,0,67,0.003265,17.922890,130.327363,184.429163
1,잠실1주차장,1059,1.044397,11,11,10,1,14.4,0,471,0.021923,144.198405,658.198578,539.334811
2,강서1주차장,329,1.361449,1,11,10,1,14.4,0,53,0.032484,342.905207,1494.617831,952.643526
3,여의도5주차장,229,1.638210,8,11,10,1,14.4,0,217,0.001260,5.267138,74.215315,72.799002
4,여의도2주차장,1934,1.293821,8,11,10,1,14.4,0,171,0.010640,44.483163,626.779124,614.817769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39780,여의도1주차장,873,4.228408,8,1,16,0,-1.7,0,462,0.002786,7.410724,126.561793,48.587604
39781,강서1주차장,28,0.206548,1,1,16,0,-1.7,0,53,0.003025,33.608470,144.567848,113.016422
39782,여의도1주차장,114,0.887865,8,1,15,0,-2.7,0,462,0.000364,0.967723,16.526969,6.344773
39783,강서1주차장,12,0.094444,1,1,15,0,-2.7,0,53,0.001296,14.403630,61.957649,48.435609
